In [1]:
import pandas as pd
import numpy as np
import pickle
import itertools

mi = 'U1'; t='PP'
# df = pd.read_csv('data/preprocessed_packing/'+mi+'.csv')
df = pd.read_csv('data/preprocessed_packing/'+mi+'_'+t+'_2.csv')# save
bbox_master = pd.read_csv('data/preprocessed_tables/'+mi+'_bbox.csv')
with open('data/preprocessed_packing/'+mi + '_'+ t +'_r.pickle', 'rb') as handle: data = pickle.load(handle)

# scaling
for i in range(len(data)): data[i] = data[i]//10
print(bbox_master.values[[0,1,2,16]]//10)
# bbox_master = np.ceil(( bbox_master.values * 0.95)/10).astype('int')
bbox_master = np.ceil(( bbox_master.values)/10).astype('int')
bbox_master = bbox_master[[0,1,2,16]]
print(bbox_master)

def get_bbox_stan(max_l, max_b, max_h, bbox_type):
    bbox_stan = np.zeros((max_l, max_b, max_h))
    l,b,h = bbox_type
    bbox_stan[l:,:,:] = 1
    bbox_stan[:,b:,:] = 1
    bbox_stan[:l, :b, :max_h-h] = 1
    return bbox_stan

def get_bbox_idx(n_bbox =4, k=2):
    # k는 박스 수
    x = list(range(4))
    idx = np.array([p for p in itertools.product(x, repeat=k)])
    idx = np.sort(idx, axis=1)
    idx = np.unique(idx, axis=0)
    return idx

def pad_bbox(n_boxes, bbox_stan, max_l, max_b, max_h, i=0):
    padded = np.ones(( max_l, max_b, max_h, n_boxes ))
    padded[:max_l, :max_b, :max_h, i] = bbox_stan
    return padded

def pad_bbox_multi(n_boxes, bbox_stans, max_l, max_b, max_h):
    padded = np.ones(( max_l, max_b, max_h, n_boxes ))
    for i in range(len(bbox_stans)):
        padded[:max_l, :max_b, :max_h, i] = bbox_stans[i]
    return padded

def merge_bboxes(bboxes):
    l, b, h, n = bboxes.shape
    n = n//2
    merged = np.zeros(( l*n+1, b*n+1, h))
    merged = merged + h
    merged[:l, :b] = bboxes[:,:,:,0]
    merged[l+1:, :b] = bboxes[:,:,:,1]
    merged[:l, b+1:] = bboxes[:,:,:,2]
    merged[l+1:, b+1:] = bboxes[:,:,:,3]
    return merged

[[119 113  50]
 [148 113 110]
 [119 113  36]
 [198 114  61]]
[[119 113  50]
 [148 113 110]
 [119 113  36]
 [198 114  61]]


In [2]:
max_l = np.max(bbox_master[:,0])
max_b = np.max(bbox_master[:,1])
max_h = np.max(bbox_master[:,2])
n_boxes = 4

max_l, max_b, max_h

(198, 114, 110)

In [3]:
bboxes1 = []
b = bbox_master[0]
bbox_stan = get_bbox_stan(max_l, max_b, max_h, b)

In [7]:
b

array([119, 113,  50])

In [4]:
bbox_stan.shape

(198, 114, 110)

In [8]:
padded = pad_bbox(n_boxes, bbox_stan, max_l, max_b, max_h, i=0)

In [9]:
padded.shape

(198, 114, 110, 4)

In [11]:
bbox_stan

array([[[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.]],

       ...,

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1.

In [10]:
padded

array([[[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         ...,
         [0., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         ...,
         [0., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         ...,
         [0., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]],

        ...,

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         ...,
         [0., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         ...,
         [0., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         ...,
 